In [2]:
%reload_ext jetisu.query_idr_magic

# Australian Capital Territory (ACT) Conveyance Duty

The following is an intensionally defined relation that captures the
relationship between price and [conveyance duty in the ACT from 1 July
2022](https://www.revenue.act.gov.au/duties/conveyance-duty). The duty
payable depends firstly on whether the purchase is non-commercial or
commercial and secondly, if non-commercial, whether the purchase is by
an owner occupier. The definition is written in
[MiniZinc](https://www.minizinc.org/).

Callouts from this example include:

  - the effort in capturing this calculation is of the same order as
    using a procedural or functional language, however the
    `ACT_Conveyance_Duty` intensionally defined relation has a bigger
    "pay-off". It is queryable in any direction, just like an ordinary
    database table

  - the usage of an intensionally defined relation `units` in the
    definition of `ACT_Conveyance_Duty`. This is a reusable relation
    that captures the idea of converting a value to a number of units, whether full or partial.

The the `ACT_Conveyance_Duty` intensionally defined relation has the
following attributes

```

    ACT_Conveyance_Duty(var bool: non_commercial,
                        var bool: eligible_owner_occupier,
                        var int:price,
                        var float:duty)
```

and the full definition in MiniZinc is shown at the end of this
appendix.


## Querying the Conveyancing Rules

Here is a selection of queries that demonstrate the number of different
insights the intensionally defined relation can provide.

### How much duty for a home occupier on a house of $1.2M?

This query gives the answer of $47,590.


In [3]:
%%jetisu_query
select price, duty from ACT_Conveyance_Duty where price = 1200000 and eligible_owner_occupier;

|price|duty|
|----|----|
|1200000.0|47590|

### How much duty for a non\_commercial property valued at of $1.2M?

This query shows that there are two classes of non-commercial property transactions, and the duty payable for each case.


In [4]:
%%jetisu_query
select eligible_owner_occupier, duty
    from ACT_Conveyance_Duty
    where price = 1200000 and non_commercial;

|eligible_owner_occupier|duty|
|----|----|
|True|47590|
|False|49750|

### How much duty is chargeable for a property valued at $1.2M?

This query shows the comparative conveyancing duty across the various pricing
regimes. Notice that the IDR has deduced and is reporting that there are
*three* categories of payers.

In [5]:
%%jetisu_query

select * from ACT_Conveyance_Duty where price = 1200000;


|non_commercial|eligible_owner_occupier|price|duty|
|----|----|----|----|
|False|False|1200000.0|0|
|True|True|1200000.0|47590|
|True|False|1200000.0|49750|

### How much duty is chargeable for a property valued at $2M?

This query shows the convergence of the owner occupied and non-occupied regimes at
higher prices.

In [6]:
%%jetisu_query

select * from ACT_Conveyance_Duty where price = 2000000;


|non_commercial|eligible_owner_occupier|price|duty|
|----|----|----|----|
|True|True|2000000.0|90800|
|True|False|2000000.0|90800|
|False|False|2000000.0|100000|

### An owner occupier pays $50,150 in conveyancing duty. What is the corresponding house price?

This query shows that there is more than one possible price that gives a duty value
of $50,150. The IDR has deduced that there are 100 such values and
reports them.

In [7]:
%%jetisu_query

select price from ACT_Conveyance_Duty where duty = 50150 and eligible_owner_occupier;


|price|
|----|
|1240000|
|1239901|
|1239902|
|1239903|
|1239904|
|1239905|
|1239906|
|1239907|
|1239908|
|1239909|
|1239910|
|1239911|
|1239912|
|1239913|
|1239914|
|1239915|
|1239916|
|1239917|
|1239918|
|1239919|
|1239920|
|1239921|
|1239922|
|1239923|
|1239924|
|1239925|
|1239926|
|1239927|
|1239928|
|1239929|
|1239930|
|1239931|
|1239932|
|1239933|
|1239934|
|1239935|
|1239936|
|1239937|
|1239938|
|1239939|
|1239940|
|1239941|
|1239942|
|1239943|
|1239944|
|1239945|
|1239946|
|1239947|
|1239948|
|1239949|
|1239950|
|1239951|
|1239952|
|1239953|
|1239954|
|1239955|
|1239956|
|1239957|
|1239958|
|1239959|
|1239960|
|1239961|
|1239962|
|1239963|
|1239964|
|1239965|
|1239966|
|1239967|
|1239968|
|1239969|
|1239970|
|1239971|
|1239972|
|1239973|
|1239974|
|1239975|
|1239976|
|1239977|
|1239978|
|1239979|
|1239980|
|1239981|
|1239982|
|1239983|
|1239984|
|1239985|
|1239986|
|1239987|
|1239988|
|1239989|
|1239990|
|1239991|
|1239992|
|1239993|
|1239994|
|1239995|
|1239996|
|1239997|
|1239998|
|1239999|

### A property owner pays $140,740 in conveyancing duty. What are the corresponding property prices?

This query shows the price ranges across the three regimes.

In [8]:
%%jetisu_query

select non_commercial,
    eligible_owner_occupier,
    min(price) min_price,
    max(price) max_price
from ACT_Conveyance_Duty
where duty = 140740
group by non_commercial,
    eligible_owner_occupier;


|non_commercial|eligible_owner_occupier|min_price|max_price|
|----|----|----|----|
|False|False|2814701|2814800|
|True|False|3099901|3100000|
|True|True|3099901|3100000|

### Definition
Here is the definition of the ```ACT_Conveyance_Duty``` intensionally defined relation.

In [9]:
%%jetisu_show
ACT_Conveyance_Duty

```

predicate units(var int:amount, 
                var int:unit_size, 
                var bool: round_up, 
                var int:units ) = 
let {
  constraint units = amount div unit_size + if round_up then amount mod unit_size > 0 else 0 endif;
} in true;



predicate act_conveyance_duty(var bool: non_commercial, 
                              var bool: eligible_owner_occupier, 
                              var int:price, 
                              var float:duty) = 
let {
  constraint eligible_owner_occupier -> non_commercial;
  constraint price >= 0;
  constraint duty >= 0;
  var int: num_units;
  int: unit = 100;
  constraint duty = 
  if non_commercial then
    if eligible_owner_occupier then
      if price > 0       /\ price <=260000  
          /\ units(price, unit, true, num_units) then num_units * 60 else 
      if price > 260000  /\ price <=300000  
          /\ units(price-260000, unit, true, num_units)  then 1560.00  + num_units * 2.20 else 
      if price > 300000  /\ price <=500000  
          /\ units(price-300000, unit, true, num_units)  then 2440.00  + num_units * 3.40 else 
      if price > 500000  /\ price <=750000  
          /\ units(price-500000, unit, true, num_units)  then 9240.00  + num_units * 4.32 else 
      if price > 750000  /\ price <=1000000 
          /\ units(price-750000, unit, true, num_units)  then 20040.00 + num_units * 5.90 else 
      if price > 1000000 /\ price <=1455000 
          /\ units(price-1000000, unit, true, num_units) then 34790.00 + num_units * 6.40 else 
      if price > 1445000                    
          /\ units(price, unit, true, num_units) then num_units * 4.54 
      else -1 endif endif endif endif endif endif endif
    else % not eligible_owner_occupier
      if price > 0       /\ price <=200000  
          /\ units(price, unit, true, num_units)         then num_units * 120 else 
      if price > 200000  /\ price <=300000  
          /\ units(price-200000, unit, true, num_units)  then 2400.00  + num_units * 2.20 else 
      if price > 300000  /\ price <=500000  
          /\ units(price-300000, unit, true, num_units)  then 4600.00  + num_units * 3.40 else 
      if price > 500000  /\ price <=750000  
          /\ units(price-500000, unit, true, num_units)  then 11400.00 + num_units * 4.32 else 
      if price > 750000  /\ price <=1000000 
          /\ units(price-750000, unit, true, num_units)  then 22200.00 + num_units * 5.90 else 
      if price > 1000000 /\ price <=1455000 
          /\ units(price-1000000, unit, true, num_units) then 36950.00 + num_units * 6.40 else 
      if price > 1445000                    
          /\ units(price, unit, true, num_units)         then num_units * 4.54 
      else -1 endif endif endif endif endif endif endif
    endif
  else % commercial
    if price > 0         /\ price <= 1700000 then 0 else
    if price > 1700000                      
          /\ units(price, unit, true, num_units)         then num_units * 5.00 else -1 endif endif
  endif
} 
in true;
```